## Análisis Exploratorio de Datos (EDA)

In [17]:
# Importar librerías necesarias
import pandas as pd
import sqlite3 as sql

### Cargar datos

In [18]:
import pandas as pd
import sqlite3 as sql

ruta_archivo = "../data/raw/adidas_us_sales_dataset.xlsx"
df = pd.read_excel(ruta_archivo, index_col=None)

# Eliminar columna de índice anterior si existe
if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'], inplace=True)

# Guardar en SQLite
conn = sql.connect("../data/raw/adidas_sales.db")
df.to_sql("ventas_adidas", conn, if_exists="replace", index=False)

print("Base de datos creada con éxito")

cursor = conn.cursor()


Base de datos creada con éxito


## Tamaño del Dataset

In [19]:
df

,Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
0,Foot Locker,1185732,1969-12-31 19:00:43.831,Northeast,New York,New York,Men's Street Footwear,50.0,1200,600000.0,300000.00,0.50,In-store
1,Foot Locker,1185732,1969-12-31 19:00:43.832,Northeast,New York,New York,Men's Athletic Footwear,50.0,1000,500000.0,150000.00,0.30,In-store
2,Foot Locker,1185732,1969-12-31 19:00:43.833,Northeast,New York,New York,Women's Street Footwear,40.0,1000,400000.0,140000.00,0.35,In-store
3,Foot Locker,1185732,1969-12-31 19:00:43.834,Northeast,New York,New York,Women's Athletic Footwear,45.0,850,382500.0,133875.00,0.35,In-store
4,Foot Locker,1185732,1969-12-31 19:00:43.835,Northeast,New York,New York,Men's Apparel,60.0,900,540000.0,162000.00,0.30,In-store
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9643,Foot Locker,1185732,1969-12-31 19:00:44.220,Northeast,New Hampshire,Manchester,Men's Apparel,50.0,64,3200.0,896.00,0.28,Outlet
9644,Foot Locker,1185732,1969-12-31 19:00:44.220,Northeast,New Hampshire,Manchester,Women's Apparel,41.0,105,4305.0,1377.60,0.32,Outlet
9645,Foot Locker,1185732,1969-12-31 19:00:44.249,Northeast,New Hampshire,Manchester,Men's Street Footwear,41.0,184,7544.0,2791.28,0.37,Outlet
9646,Foot Locker,1185732,1969-12-31 19:00:44.249,Northeast,New Hampshire,Manchester,Men's Athletic Footwear,42.0,70,2940.0,1234.80,0.42,Outlet


Tamaño del dataset: 9648 Filas y 13 columnas

## Tipos de variables detectadas

In [29]:
df.dtypes

Retailer                    object
Retailer ID                  int64
Invoice Date        datetime64[ns]
Region                      object
State                       object
City                        object
Product                     object
Price per Unit             float64
Units Sold                   int64
Total Sales                float64
Operating Profit           float64
Operating Margin           float64
Sales Method                object
dtype: object

#### Numéricas:

- Retailer ID
- Price per Unit
- Units Sold
- Total Sales
- Operating Profit
- Operating Margin

#### Categóricas:

- Retailer
- Region
- State
- City
- Product
- Sales Method

#### Fechas:

- Invoice Date

## Exploración Inicial y calidad de datos:

### Columnas con valores nulos:

In [22]:
valores_nulos = """
                SELECT
                    SUM(CASE WHEN Retailer IS NULL THEN 1 ELSE 0 END) AS retailer_nulos,
                    SUM(CASE WHEN `Retailer ID` IS NULL THEN 1 ELSE 0 END) AS retailerid_nulos,
                    SUM(CASE WHEN `Invoice Date` IS NULL THEN 1 ELSE 0 END) AS invoice_date_nulos,
                    SUM(CASE WHEN Region IS NULL THEN 1 ELSE 0 END) AS region_nulos,
                    SUM(CASE WHEN State IS NULL THEN 1 ELSE 0 END) AS state_nulos,
                    SUM(CASE WHEN City IS NULL THEN 1 ELSE 0 END) AS city_nulos,
                    SUM(CASE WHEN Product IS NULL THEN 1 ELSE 0 END) AS products_nulos,
                    SUM(CASE WHEN `Price per Unit` IS NULL THEN 1 ELSE 0 END) AS price_per_unit_nulos,
                    SUM(CASE WHEN `Units Sold` IS NULL THEN 1 ELSE 0 END) AS units_sold_nulos,
                    SUM(CASE WHEN `Total Sales` IS NULL THEN 1 ELSE 0 END) AS total_sales_nulos,
                    SUM(CASE WHEN `Operating Profit` IS NULL THEN 1 ELSE 0 END) AS operating_profit_nulos,
                    SUM(CASE WHEN `Operating Margin` IS NULL THEN 1 ELSE 0 END) AS operating_margin_nulos,
                    SUM(CASE WHEN `Sales Method` IS NULL THEN 1 ELSE 0 END) AS sales_methond_nulos
                FROM ventas_adidas;
                """
valores_nulos = pd.read_sql_query(valores_nulos, conn)
valores_nulos

,retailer_nulos,retailerid_nulos,invoice_date_nulos,region_nulos,state_nulos,city_nulos,products_nulos,price_per_unit_nulos,units_sold_nulos,total_sales_nulos,operating_profit_nulos,operating_margin_nulos,sales_methond_nulos
0,0,0,0,0,0,0,0,0,0,0,0,0,0


En conclusión no hay valores nulos

### Registros duplicados